<a href="https://colab.research.google.com/github/ipietri/w266_Final_Project/blob/master/notebooks/RtGender-Notebooks/Greyscaling/RtGender_Sentiment_Greyscaling_Universal_Sentence_Encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

[link text](https://)# Universal Sentence Encoder


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
  <td>
    <a href="https://tfhub.dev/s?q=google%2Funiversal-sentence-encoder%2F4%20OR%20google%2Funiversal-sentence-encoder-large%2F5"><img src="https://www.tensorflow.org/images/hub_logo_32px.png" />See TF Hub models</a>
  </td>
</table>

This notebook illustrates how to access the Universal Sentence Encoder and use it for sentence similarity and sentence classification tasks.

The Universal Sentence Encoder makes getting sentence level embeddings as easy as it has historically been to lookup the embeddings for individual words. The sentence embeddings can then be trivially used to compute sentence level meaning similarity as well as to enable better performance on downstream classification tasks using less supervised training data.


## Setup

This section sets up the environment for access to the Universal Sentence Encoder on TF Hub and provides examples of applying the encoder to words, sentences, and paragraphs.

In [ ]:
%%capture
!pip3 install seaborn

More detailed information about installing Tensorflow can be found at [https://www.tensorflow.org/install/](https://www.tensorflow.org/install/).

In [1]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
#@title Compute a representation for each message, showing various lengths supported.
word = "Elephant"
sentence = "I am a sentence for which I would like to get its embedding."
paragraph = (
    "Universal Sentence Encoder embeddings also support short paragraphs. "
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, paragraph]

# Reduce logging output.
logging.set_verbosity(logging.ERROR)

message_embeddings = embed(messages)

for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
  print("Message: {}".format(messages[i]))
  print("Embedding size: {}".format(len(message_embedding)))
  message_embedding_snippet = ", ".join(
      (str(x) for x in message_embedding[:3]))
  print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

# Semantic Textual Similarity Task Example

The embeddings produced by the Universal Sentence Encoder are approximately normalized. The semantic similarity of two sentences can be trivially computed as the inner product of the encodings.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Similarity

In [4]:
train_df_aug = pd.read_csv('/content/drive/MyDrive/w266/grey_scaled_augmented_oversampled_rtgender_train_data.csv')
train_df_aug['list_text'] = train_df_aug['text'].apply(lambda x: x.split())

In [5]:
import math

# get each unique id and then cycle through
unique_id_list = train_df_aug['id'].unique().tolist()

# instantiate lists 
scores_list = []
cs_list = []
mean_list = []
og_sentence = []
augmented_sentence_list = []
id_list = []

# for every origin
for id in unique_id_list: 
  # subset the dataframe and separate original and augmented examples
  data = train_df_aug[train_df_aug['id'] == id]
  og = data[data['is_og'] == 1].iloc[0].text
  og_list = data[data['is_og'] == 1].iloc[0].list_text
  
  # return all augmented examples
  augmented = data[data['is_og'] == 0]

  # if there are no augmented examples add NA to list
  if len(augmented) == 0:
    id_list.append(id)
    scores_list.append(np.nan)
    cs_list.append(np.nan)
    mean_list.append(np.nan)
    og_sentence.append(og)
    augmented_sentence_list.append(np.nan)
    

  else: 
    # encode the original example; requires list format
    sts_encode1 = tf.nn.l2_normalize(embed(tf.constant(og_list)), axis=1)
    for i in range(len(augmented)):
      example = augmented.iloc[i].text
      example_list_text = augmented.iloc[i].list_text
      # add the score for each augmented example to scores_list
      sts_encode2 = tf.nn.l2_normalize(embed(tf.constant(example_list_text)), axis=1)
      
      #cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
      # encode1 = embed(tf.constant(og))
      # encode2 = embed(tf.constant(example))
      m = tf.keras.metrics.CosineSimilarity(axis = 1)
      m.update_state(sts_encode1, sts_encode2)
      
      cosine_similarities = m.result().numpy()
      clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
      scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi

      id_list.append(id)
      og_sentence.append(og)
      augmented_sentence_list.append(example)
      cs_list.append(cosine_similarities)
      scores_list.append(scores.numpy())
      
      mean_score = tf.reduce_mean(scores).numpy()
      mean_list.append(mean_score)


In [6]:
df_with_scores = pd.DataFrame({'id':id_list, 'og_sentence':og_sentence, 
              'augmented_sentence':augmented_sentence_list,
              'mean_arc_sine_score': mean_list})

df_with_scores

,id,og_sentence,augmented_sentence,mean_arc_sine_score
0,0,Thank you Congresswoman Bass. Keep up the grea...,Thank you Congresswoman Bass. Keep up the good...,0.941539
1,5,I'm looking forward to seeing some *Good News*...,I'm looking forward to seeing some *Good News*...,0.958211
2,5,I'm looking forward to seeing some *Good News*...,I'm looking forward to seeing some *Good News*...,0.960997
3,5,I'm looking forward to seeing some *Good News*...,I'm looking forward to seeing some *Good News*...,0.961768
4,5,I'm looking forward to seeing some *Good News*...,I'm looking forward to seeing some *Good News*...,0.957842
...,...,...,...,...
35478,21179,"""Committed to making sure we don't lose our he...",NaN,NaN
35479,21180,Both Isakson and Chambliss voted to TABLE Rand...,NaN,NaN
35480,21181,I think a magic beam of pure light would disin...,NaN,NaN
35481,21182,I'd rather have a root canal . . .,NaN,NaN


In [7]:
df_with_scores.to_csv('/content/drive/MyDrive/w266/universal_sentence_arc_df_rtgender.csv', index = False)

Visualize distribution

In [8]:
mean_arc_sine_score = df_with_scores['mean_arc_sine_score'].to_list()

[Text(0.5, 1.0, 'RtGender Mean Arc Sine Score')]

In [9]:
sns.histplot(df_with_scores['mean_arc_sine_score']).set(title='RtGender Mean Arc Sine Score')

The data are not normally distributed. Approximately 25% of our examples have an arc sine of less than 0.98. 

In [10]:
print("P10: ", np.nanquantile(mean_arc_sine_score, 0.1))
print("P25: ", np.nanquantile(mean_arc_sine_score, 0.25))
print("P50: ", np.nanquantile(mean_arc_sine_score, 0.5))
print("P75: ", np.nanquantile(mean_arc_sine_score, 0.75))
print("P90: ", np.nanquantile(mean_arc_sine_score, 0.9))

p25_threshold = np.nanquantile(mean_arc_sine_score, 0.25)

P10:  0.9010409712791443
P25:  0.9261012673377991
P50:  0.9441190958023071
P75:  0.9562197327613831
P90:  0.9649546146392822


Low match examples

In [11]:
# standard deviation ignoring NAs
np.nanstd(df_with_scores['mean_arc_sine_score'])

0.027158550558489994

In [12]:
df_with_scores[df_with_scores['mean_arc_sine_score'] < .95]

,id,og_sentence,augmented_sentence,mean_arc_sine_score
0,0,Thank you Congresswoman Bass. Keep up the grea...,Thank you Congresswoman Bass. Keep up the good...,0.941539
12,6,"Yay for useful, practical presi's!! :D","Yay for useful, possible presi's!! :D",0.852176
13,6,"Yay for useful, practical presi's!! :D","Yay for useful, realistic presi's!! :D",0.864039
14,6,"Yay for useful, practical presi's!! :D","Yay for useful, feasible presi's!! :D",0.871951
15,8,"Actually, they looked dopey, but they did pret...","Actually, they looked dopey, but they did hand...",0.913076
...,...,...,...,...
20980,21159,Bad decision to go on recess while we starve h...,mediocre decision to go on recess while we sta...,0.923667
20983,21165,"Im from Australia and as bad as we have it, br...","Im from Australia and as cheap as we have it, ...",0.940893
20984,21165,"Im from Australia and as bad as we have it, br...",Im from Australia and as mediocre as we have i...,0.947099
20985,21166,"many already had coverage, with doctors and pl...","some already had coverage, with doctors and pl...",0.947972


Strong Match Examples

In [13]:
strong_matches = df_with_scores[df_with_scores['mean_arc_sine_score'] > .994]

In [14]:
strong_matches[strong_matches['id'] == 521]

,id,og_sentence,augmented_sentence,mean_arc_sine_score


# Pre-process 

In [16]:
df_aug = pd.read_csv('/content/drive/MyDrive/w266/universal_sentence_arc_df_rtgender.csv')

Keep all original text plus all augmented examples that meet or exceed the 25th %tile

In [18]:
# set threshold at P25 
p25_threshold = np.nanquantile(df_aug.mean_arc_sine_score, 0.25)
print("threshold for similarity: ", p25_threshold)


threshold for similarity:  0.9261012673377992


In [19]:
augmented_examples_to_remove = df_aug[df_aug['mean_arc_sine_score']< p25_threshold]['augmented_sentence'].to_list()
train_df_aug_rc = train_df_aug[~train_df_aug['text'].isin(augmented_examples_to_remove)]
train_df_aug_rc.to_csv('/content/drive/MyDrive/w266/grey_scaled_augmented_oversampled_rtgender_train_data_contextually_representative_ex_only.csv', index = False)



## Import Dev Data and Convert to HuggingFace friendly format

In [20]:
train_df = train_df_aug_rc
dev_df = pd.read_csv('/content/drive/MyDrive/w266/annotations_dev.csv')
print('dev_shape: ',dev_df.shape)

# there are NaNs in the dev dataset remove 
nan_values = dev_df[dev_df.isna().any(axis=1)] 
print(nan_values)

# return without missing values in response_text
dev_df.dropna(subset = ["response_text"], inplace=True)

print("Train shape", train_df.shape)
print("Dev shape", dev_df.shape)

dev_shape:  (2303, 9)
      Unnamed: 0         source op_gender  ...   relevance label labels_4
830         2576  facebook_wiki         M  ...  Irrelevant     1        1
1664        2722  facebook_wiki         W  ...  Irrelevant     1        1

[2 rows x 9 columns]
Train shape (36939, 6)
Dev shape (2301, 9)


In [21]:
# keep only a few columns and rename dev col names
dev_df = dev_df[['response_text', 'labels_4']]
dev_df.rename(columns = {'response_text':'text', 'labels_4': 'label'}, inplace = True)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Huggingface friendly format

In [22]:
%%capture
!pip install datasets
import datasets 
from datasets import load_dataset, Dataset, DatasetDict

In [23]:
# change to dataset to work with Huggingface transformer & remove unused columns
columns_to_remove = ['id', '__index_level_0__']

train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)

train_dataset = train_dataset.remove_columns(column_names= columns_to_remove)
dev_dataset = dev_dataset.remove_columns(column_names= '__index_level_0__')

# combine into a DataDictionary for huggingface use
rtg_dataset = DatasetDict({
    'train': train_dataset,
    'dev': dev_dataset 
})

rtg_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'is_og', 'new_col', 'list_text'],
        num_rows: 36939
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 2301
    })
})

# Tokenize 

In [24]:
%%capture
!pip install -q transformers
import transformers
import json
from tqdm import tqdm
#from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import logging
logging.basicConfig(level=logging.ERROR)

# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [25]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
max_length = train_df['text'].astype(str).map(len).quantile(0.99).astype(int)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_length = int(max_length))

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

rtg_encoded = rtg_dataset.map(tokenize, batched=True, batch_size=None)


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

# Model

In [26]:
from transformers import AutoModelForSequenceClassification
num_labels = 4
epochs = 2
iterations = 5
model = (AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device))

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

In [27]:
rtg_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [28]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1_weighted = f1_score(labels, preds, average="weighted")
    f1_macro = f1_score(labels, preds, average = 'macro')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1_weighted, "f1_macro": f1_macro} 

In [29]:
from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(rtg_encoded["train"]) // batch_size*3
training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=epochs,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  load_best_model_at_end=True,
                                 # metric_for_best_model="f1_macro",
                                 # weight_decay=0.01,
                                  evaluation_strategy="steps",
                                  save_strategy="steps",
                                  disable_tqdm=False
                                  )

In [30]:
from sklearn.metrics import classification_report

accuracy_list = []
weighted_f1_score_list = []
macro_f1_score_list = []
negative_f1_score = []
neutral_f1_score = []
mixed_f1_score = []
positive_f1_score = []


for i in range(iterations):
  try:
    del trainer
    del results
    del cr
  except: pass


  trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=rtg_encoded["train"],
                  eval_dataset=rtg_encoded["dev"])
  trainer.train()
  results = trainer.evaluate()

  # append macro metrics to lists
  accuracy_list.append(results.get('eval_accuracy'))
  weighted_f1_score_list.append(results.get("eval_f1"))
  macro_f1_score_list.append(results.get("eval_f1_macro"))

  trainer.predict(rtg_encoded["dev"])
  # append the class-level F1 scores
  outputs = trainer.predict(rtg_encoded["dev"])
  predictions = outputs.predictions.argmax(1)
  labels = rtg_encoded["dev"]['label']
  cr = classification_report(labels, predictions, digits=3, output_dict=True)
  negative_f1_score.append(cr.get('0').get("f1-score"))
  neutral_f1_score.append(cr.get('1').get("f1-score"))
  positive_f1_score.append(cr.get('2').get("f1-score"))
  mixed_f1_score.append(cr.get('3').get("f1-score"))


  print(f'---------------------------Iteration {i+1} Complete---------------------------\n')

Step,Training Loss,Validation Loss,Accuracy,F1,F1 Macro
500,1.057500,0.974426,0.619731,0.634148,0.533899
1000,0.796500,0.913638,0.669274,0.666277,0.556688
1500,0.664000,1.074428,0.645372,0.654271,0.548101
2000,0.566500,1.021544,0.656671,0.666808,0.566013
2500,0.496100,1.072016,0.655802,0.661679,0.562239
3000,0.455200,1.102299,0.656236,0.657306,0.546249
3500,0.393400,1.311926,0.662755,0.664869,0.558647
4000,0.373100,1.375800,0.668405,0.661906,0.553161
4500,0.325700,1.530241,0.643199,0.647845,0.545271
5000,0.235200,1.683847,0.668840,0.668349,0.565636


---------------------------Iteration 1 Complete---------------------------



Step,Training Loss,Validation Loss,Accuracy,F1,F1 Macro
500,0.389100,1.698650,0.601912,0.629340,0.535511
1000,0.280600,1.656044,0.645372,0.646498,0.536208
1500,0.606500,1.225978,0.640156,0.657254,0.553717
2000,0.497200,1.146963,0.642764,0.654467,0.554843
2500,0.442900,1.178549,0.640591,0.650591,0.550489
3000,0.400100,1.240795,0.653629,0.656084,0.547983
3500,0.349300,1.446439,0.654063,0.660749,0.558934
4000,0.328700,1.586371,0.657975,0.648250,0.534638
4500,0.304600,1.591090,0.654933,0.660212,0.559084
5000,0.195300,1.859882,0.654498,0.659935,0.556396


---------------------------Iteration 2 Complete---------------------------



Step,Training Loss,Validation Loss,Accuracy,F1,F1 Macro
500,0.130500,1.988693,0.654063,0.653530,0.547155
1000,0.114100,2.257935,0.631030,0.639536,0.534647
1500,0.207400,2.045093,0.611473,0.635442,0.540474
2000,0.149500,2.146941,0.621034,0.637042,0.537214
2500,0.442200,1.417808,0.626249,0.640538,0.540135
3000,0.379500,1.376410,0.654933,0.654164,0.541435
3500,0.329100,1.643342,0.648848,0.648763,0.535713
4000,0.287700,1.746190,0.656236,0.654201,0.544649
4500,0.288700,1.764450,0.655367,0.655145,0.550272
5000,0.162900,2.060883,0.663625,0.663617,0.559872


---------------------------Iteration 3 Complete---------------------------



Step,Training Loss,Validation Loss,Accuracy,F1,F1 Macro
500,0.056600,2.613347,0.641895,0.655042,0.556003
1000,0.067900,2.533902,0.661886,0.654598,0.543745
1500,0.101900,2.519482,0.638853,0.639354,0.532037
2000,0.083400,2.936627,0.601043,0.616657,0.513777
2500,0.184400,2.477994,0.615819,0.634315,0.537281
3000,0.161800,2.075050,0.640156,0.641532,0.535575
3500,0.325000,1.865366,0.650152,0.644410,0.535923
4000,0.288800,1.957993,0.645806,0.648612,0.542520
4500,0.274600,1.856261,0.649283,0.653999,0.549868
5000,0.137600,2.242601,0.657106,0.653958,0.551422


---------------------------Iteration 4 Complete---------------------------



Step,Training Loss,Validation Loss,Accuracy,F1,F1 Macro
500,0.049100,2.729551,0.655802,0.651472,0.544128
1000,0.033600,3.033541,0.633203,0.645829,0.546263
1500,0.047300,2.700348,0.649283,0.649670,0.546724
2000,0.048000,2.870866,0.643633,0.630282,0.524948
2500,0.096200,2.636498,0.649718,0.641672,0.525023
3000,0.081900,2.490075,0.641026,0.642438,0.532283
3500,0.138300,2.410071,0.657975,0.643232,0.533595
4000,0.133600,2.367921,0.650152,0.638339,0.527585
4500,0.124900,2.298015,0.641895,0.645602,0.540173
5000,0.130300,2.514395,0.646241,0.645061,0.538353


---------------------------Iteration 5 Complete---------------------------



# Evaluate BERT

In [31]:
import statistics

print("%15s %s (%s)" % ("","Mean", "StDev"))

print("-"*29)
print("Macro Scores")
print("-"*29)

print(f"%15s %s (%s)" %("Accuracy",
    round(statistics.mean(accuracy_list),3),
    round(statistics.stdev(accuracy_list),3)))
print(f"%15s %5s (%s)" %("Macro F1",
    round(statistics.mean(macro_f1_score_list),3),
    round(statistics.stdev(macro_f1_score_list),3)))
print(f"%15s %5s (%s)" %("Weighted F1",
    round(statistics.mean(weighted_f1_score_list),3),
    round(statistics.stdev(weighted_f1_score_list),3)))

print("-"*29)
print("Class Scores")
print("-"*29)

print(f"%15s %s (%s)" %("Positive",
    round(statistics.mean(positive_f1_score),3),
    round(statistics.stdev(positive_f1_score),3)))
print(f"%15s %5s (%s)" %("Neutral",
    round(statistics.mean(neutral_f1_score),3),
    round(statistics.stdev(neutral_f1_score),3)))
print(f"%15s %5s (%s)" %("Negative",
    round(statistics.mean(negative_f1_score),3),
    round(statistics.stdev(negative_f1_score),3)))
print(f"%15s %5s (%s)" %("Mixed",
    round(statistics.mean(mixed_f1_score),3),
    round(statistics.stdev(mixed_f1_score),3)))

                Mean (StDev)
-----------------------------
Macro Scores
-----------------------------
       Accuracy 0.652 (0.011)
       Macro F1 0.549 (0.008)
    Weighted F1 0.655 (0.007)
-----------------------------
Class Scores
-----------------------------
       Positive 0.811 (0.009)
        Neutral 0.581 (0.012)
       Negative 0.571 (0.011)
          Mixed 0.231 (0.025)
